# LISP Interpreter in Python 3

Python 3으로 작성된 LISP Interpreter에 대한 Program Document입니다.  
코드의 설명을 위해 본 문서는 Jupyyer Notebook으로 작성되었으나, 실제 Interpreter는 .py를 통해서도 원활하게 구동됩니다.

*** 
**[수강 정보]**
- 과목/분반: 프로그래밍언어론 02분반
- 담당 교수: 이남규 교수님

**[Team 20]**   
- 20194538 이나혁 (소프트웨어학부 2학년)  
- 20193418 이하윤 (소프트웨어학부 2학년)     
- 20192939 이하은 (소프트웨어학부 2학년)  

**[작성일]** 2020월 12월 05일

**[구현환경]**
- Environment1 (이나혁) : macOS Catalina / Python 3.7.6 (with Anaconda), Pycharm Professional 2020.1.2, Jupyter Notebook
- Environment2 (이하윤) : Windows 10 / Python 3.8.3, Microsoft Visual Studio Code with Python
- Environment3 (이하은) : Windows 10 / Python 3.6.6, Microsoft Visual Studio Code with Python

***
## Import Package 

패키지 사용은 Python 3에서 기본적으로 제공하는 **내장 라이브러리만 사용**하는 것을 원칙으로 하였다.  
LISP Interpreter 구현을 위해 사용한 패키지와 그 목적에 대한 서술은 아래와 같다.

- **math** : 기본 수학 계산을 위해 사용됨.
- **functools** : reduce()의 사용을 위함.  
    * **reduce()**는 가변 인자에 대한 수학 연산에 사용됨.
- **operator** : 수학 연산자 사용을 위함.
- **fractions** : Fraction()의 사용을 위함.
    * **Fraction()**은 int형과 float형을 분수화하는데 사용됨.

In [1]:
import math
from functools import reduce
import operator as op
from fractions import Fraction

***
## LISP 자료형 정의

LISP 1.5 표준에 대한 CFG(Context-Free Grammar)는 아래와 같이 표현된다.  
(출처: https://link.springer.com/chapter/10.1007/978-3-319-27261-0_22)

![LISP EBNF](https://media.springernature.com/lw785/springer-static/image/chp%3A10.1007%2F978-3-319-27261-0_22/MediaObjects/394229_1_En_22_Figa_HTML.gif)

우리는 LISP을 Python에서 다루기 위해 LISP에 대한 자료형을 Python에서 정의해야만 한다.

- **LISP's Number** : Python의 int와 float가 대응된다.
- **LISP's Letter** : Python에는 문자형 자료형이 존재하지 않는다. 때문에 이 구현은 String 영역에서 해결할 수 있다.
- **LISP's Atom** : Atom은 위에 선언된 자료형인 Symbol과 Number로 구성된다. 또한 구성의 순서도 상관 없다.
- **LISP's List** : Python의 list 자료구조와 대응된다.
- **LISP's Expression** : LISP의 Expression은 위에 선언된 자료형인 Atom과 List로 구성된다.

In [2]:
Number = (int, float)
Letter = str
Atom = (Letter, Number)
List = list
Exp = (Atom, List)

**Env**는 LISP의 함수 및 연산자, 그리고 전역 변수의 심볼과 그 값을 저장하는데 사용되며, Python의 Dictionary 자료형을 이용한다.

In [3]:
Env = dict

***
## Lexer 구현

Lexer는 lexical analysis(어휘 분석)를 수행한다. 텍스트를 입력받고 한 글자 한 글자 읽어나가다가 의미를 가진 단어를 만나면 Token을 생성한다.   

**tokenize()** 함수는 인자로 String을 받은 후 **.replace()** 메소드와 **.split()** 메소드를 활용하여 Token 단위로 분리하여 list 자료구조에 저장한다.  
string(), quote(), star() 함수는 아래 새롭게 정의하며 설명을 추가하도록 한다.

In [4]:
# LEXER 함수
def tokenize(str):
    tokens = str.replace("(", " ( ").replace(")", " ) ").replace("'", " ' ").replace('"', ' " ').replace('#',' # ').split()
    temp = quote(string(tokens))
    return star(temp)

**quote()** 함수는 다음의 두 기능을 수행한 후 전처리 된 새로운 List를 반환한다.  
check_frac() 함수는 아래 새롭게 정의하며 설명을 추가하도록 한다.

- **' Token이 주어지면 아래의 작업를 위한 전처리를 실행**
    * ' 바로 뒤 ( Token이 오면 List 인용
    * ' 바로 뒤 그 외의 Token이 오면 Symbol 인용
      
- **# Token이 주어지면 아래의 작업을 위한 전처리를 실행**
    * '#' 바로 뒤 ( Token이 오면 배열로 인식 (이 Interperter에서는 List와 배열을 동일하게 취급함.)
    * '#' 바로 뒤 \ Token이 오면 문자형 인식

In [5]:
# 인용기호 맟 '#' 처리 함수
def quote(tokens: list):
    i = 0
    n = len(tokens)
    new_list = []
    temp = ""
    count = 0
    
    while (i < n):        
        # "'" TOKEN 확인 시
        if tokens[i] == "'":
            new_list.append("(")
            new_list.append("QUOTE")
            i += 1
            
            # 다음 토큰이 "("인 경우 : 리스트 인용
            if tokens[i] == "(":
                count += 1
                while True:
                    new_list.append(tokens[i])
                    i += 1
                    if tokens[i] == "(":
                        count += 1
                    elif tokens[i] == ")":
                        count -= 1

                    if count == 0:  # ")" 까지 반복
                        new_list.append(tokens[i])
                        # new_list = []
                        break

            # 다음 토큰이 "("가 아닌 경우 : 심볼 인용
            else:
                new_list.append(tokens[i])
                new_list.append(")")
                i += 1

        # '#' TOKEN 확인 시
        elif tokens[i] == "#":
            
            # 배열 저장
            if tokens[i + 1] == "(":
                new_list.append("(")
                new_list.append("QUOTE")
                i += 1
                
                # 다음 토큰이 "("인 경우 : 리스트 인용
                if tokens[i] == "(":
                    count += 1
                    while True:
                        new_list.append(tokens[i])
                        i += 1
                        if tokens[i] == "(":
                            count += 1
                        elif tokens[i] == ")":
                            count -= 1

                        if count == 0:  # ")" 까지 반복
                            new_list.append(tokens[i])
                            # new_list = []
                            # new_list.append(")")
                            break

                # 다음 토큰이 "("가 아닌 경우 : 심볼 인용
                else:
                    new_list.append(tokens[i])
                    new_list.append(")")
                    i += 1
                    
            # 문자형 저장
            elif tokens[i + 1][0] == "\\":
                temp += "#"
                i += 1
                temp += tokens[i]
                new_list.append(temp)
                temp = ""
                i += 1


        # 인용기호 TOKEN이 아닐 경우
        else:
            new_list.append(check_frac(tokens[i]))
            i += 1

    return new_list

**string()** 함수는 인자로 String을 받은 후 " Token으로 묶인 Token에 대해 문자열로 인식하도록 하는 전처리를 수행한 후 새로운 List를 반환한다.

In [6]:
# 문자열 처리 함수
def string(tokens: list):
    i = 0
    n = len(tokens)
    new_list = []
    temp = ""
    
    while (i < n):
        # " TOKEN 확인 시
        if tokens[i] == '"':
            temp = temp + '"'
            i += 1
            
            # 다음 " Token 발견 시 까지 반복 수행
            while True:
                if tokens[i] == '"':
                    temp = temp + '"'
                    temp = temp[:-2] + temp[-1]
                    new_list.append(temp)
                    temp = ""
                    i += 1
                    break

                temp += tokens[i]
                temp += " "
                i += 1
                
        # 다음 토큰이 '"'가 아닌 경우 : 심볼 인용
        else:
            if tokens[i][0:1] == '\\' and tokens[i - 1] == '#':
                new_list.append(tokens[i])
            else:
                new_list.append(tokens[i].upper())
            i += 1
    return new_list

**star()** 함수는 향후 * 의 두 기능이 모두 수행이 가능하도록 사전 전처리를 수행하는 함수이다.  
- **LISP 내 * 의 기능 1** : 사칙 연산(곱셈) 기능 
- **LISP 내 * 의 기능 2** : 사전에 Return된 값을 저장하고 있는 Symbol 기능 **(추가 구현 사항)**  


구현 윈리는 인용을 위한 quote() 함수와 비슷하다.

In [7]:
def star(tokens: list):
    i = 0
    n = len(tokens)
    new_list = []

    while (i < n):
        if tokens[i] == '*':
            if i == 0:
                new_list.append('(')
                new_list.append('STAR')
                new_list.append(')')
            elif tokens[i - 1] == '(':
                new_list.append('*')
            else:
                new_list.append('(')
                new_list.append('STAR')
                new_list.append(')')
            i += 1
        else:
            new_list.append(tokens[i])
            i += 1
    return new_list

**(추가 구현 사항)** LISP는 기본적으로 연산 결과가 정수형이 아닐 시, 분수로 값을 출력한다.  단, 피연산자 중 실수형(float)이 하나라도 존재한다면 그 결과를 실수형으로 출력한다. 이 기능를 위한 전처리 함수가 **check_frac()**이다.  
이 함수는 인자로 String을 받은 후 특정 숫자를 분수화시켜주는 함수이며, 이에 Python 내장 패키지 **fractions**의 Fraction() 함수가 사용된다.

In [8]:
# 분수 확인 함수
def check_frac(token: str):
    new = token.split('/')
    # print(new)
    if len(new) == 2 and (new[0] != ''):
        try:
            f1 = int(new[0])  # 분자
            f2 = int(new[1])  # 분모
            token = str(Fraction(f1, f2))
        except:
            return token
    return token

***
## Parser 구현

재귀-하강식 파서(Recursive-descent Parser)를 구현한다.   

**read_from_tokens()** 함수는 전처리된 Token List를 읽고 Interpreter가 수행할 작업을 결정한다.  
스택(Stack) 자료구조를 이용하여 Token을 순차적으로 읽고 Pop하며 작업을 수행한다.  
atom() 함수는 아래 새롭게 정의하며 설명을 추가하도록 한다.

In [9]:
# Token 읽기 함수
def read_from_tokens(tokens: list) -> Exp:
    if len(tokens) == 0:
        raise SyntaxError('unexpected EOF')

    token = tokens.pop(0)
    if token == '(':
        if len(tokens) == 1:  # ( ) 빈 입력 들어오면 nil 출력
            return False
        L = []
        while tokens[0] != ')':
            L.append(read_from_tokens(tokens))
        tokens.pop(0)  # pop off ')'
        return L
    elif token == ')':
        raise SyntaxError('unexpected )')
    else:
        return atom(token)

Lexer로 분리된 Token들은 모두 String형으로 저장되어있다.  
**atom()** 함수는 이러한 Token들에 대해 자료형을 부여한다.

In [10]:
# Atom 자료형 부여 함수
def atom(token):
    try:
        return int(token)
    except ValueError:
        try:
            return float(token)
        except ValueError:
            return str(token)

**parse()** 함수는 사용자 input에 tokenize(), read_from_tokens() 함수를 순차적으로 적용하고 이를 리턴한다.

In [11]:
# 파싱 함수
def parse(program):
    return read_from_tokens(tokenize(program))

***
## LISP 함수 구현

재귀-하강식 파서(Recursive-descent Parser)를 구현한다.  

이 방식을 통해 복합문에 대한 효과적인 파싱을 수행할 수 있다. (인자로서의 함수, 변수(심볼) 해석 등)  

대부분의 함수는 **입력 인자의 개수가 정해져있더라도 가변인자를 받도록 설계**하였다. 그 이유는 입력 인자 개수 오류에 대한 예외 처리를 실행하기 위함이다.

***
## Specification 1 : LISP의 산술 연산

사칙 연산(+, -, *, /) 함수에 대한 정의이다. 또한 주어진 spec에는 존재하지 않지만, '='에 대한 서술을 포함한다.<b>(추가 구현 사항)</b>  
  
5가지 연산자(+, -, *, /, =)에 대해 공통되는 특징은 아래와 같다.
- 오로지 숫자형 인자만 입력받을 수 있다. 
- 가변 인자를 사용하여 정해지지 않은 수의 인자를 받을 수 있다.

LISP는 분수 표현을 지원한다. 하지만 사칙 연산 함수(+, -, *, /) 인자 중 하나라도 실수형(float) 숫자가 존재하면 결과로 실수를 반환한다.  
따라서 <b>floatCheck</b> 변수를 통해 출력 모드를 설정할 수 있다.

In [12]:
floatCheck = False

### + (ADD) 함수 정의
숫자 간 덧셈 연산을 수행하는 함수이다. 가변인자 args을 입력받고 연산 결과(int/float/Fraction)를 반환한다. 
- **인자가 0개인 경우**: 0 Return
- **인자가 1개인 경우**: (0 + 인자) Return
- **인자가 2개 이상인 경우**: 모든 인자의 합 Return

In [13]:
# ADD 정의
def ADD_func(*args):
    
    # 인자가 1개도 존재하지 않을 시
    if len(args) == 0:
        return 0

    checkFloat = False

    for i in args:
        # 심볼 안에 값 정의되지 않았을 시
        if i == None:
            return None

        # 인자 값 중 NIL 존재 하면 에러
        if type(i) == bool and i == False:
            print("[ERROR] Value", schemestr(i), "not a NUMBER.")
            return None
        
        # float형 체크
        if type(i) == float:
            checkFloat = True
            
        # Fraction 변환 가능 여부 체크
        try:
            Fraction(i)  # int, float 체크
        except:
            print("[ERROR] Value", i, "not a NUMBER.")
            return None
    
    # 정상 작동 경우
    if checkFloat == True:
        return float(reduce(op.add, args))
    else:
        return reduce(op.add, args)

### * (MUL) 함수 정의
숫자 간 곱셈 연산을 수행하는 함수이다.  가변인자 args을 입력받고 연산 결과(int/float/Fraction)를 반환한다.  
- **인자가 0개인 경우**: 1 Return
- **인자가 1개인 경우**: (1 * 인자) Return
- **인자가 2개 이상인 경우**: 모든 인자의 곱 Return

In [14]:
# MUL 정의
def MUL_func(*args):
    
    # 인자가 1개도 존재하지 않을 시
    if len(args) == 0:
        return 1

    checkFloat = False

    for i in args:
        # 심볼 안에 값 정의되지 않았을 시
        if i == None:
            return None

        # 인자 값 중 NIL 존재 하면 에러
        if type(i) == bool and i == False:
            print("[ERROR] Value", schemestr(i), "not a NUMBER.")
            return None
        
        # float형 체크
        if type(i) == float:
            checkFloat = True
            
        # Fraction 변환 가능 여부 체크
        try:
            Fraction(i)  # int, float 체크
        except:
            print("[ERROR] Value", schemestr(i), "not a NUMBER.")
            return None
        
    # 정상 작동 경우
    if checkFloat == True:
        return float(reduce(op.mul, args))
    else:
        return reduce(op.mul, args)

### - (SUB) 함수 정의
숫자 간 뺄셈 연산을 수행하는 함수이다. 인자 n과 가변인자 args을 입력받고 연산 결과(int/float/Fraction)를 반환한다.
- **인자가 0개인 경우**: Error
- **인자가 1개인 경우**: (- 인자) Return
- **인자가 2개 이상인 경우**: 첫번째 인자에서 나머지 인자들의 합을 뺄셈한 값 Return

In [15]:
# SUB 정의
def SUB_func(n, *args):
    checkFloat = False

    # 심볼 안에 값 정의되지 않았을 시
    if n == None:
        return None

    # 인자 값 중 NIL 존재 하면 에러
    if type(n) == bool and n == False:
        print("[ERROR] Value", schemestr(n), "not a NUMBER.")
        return None

    # float형 체크
    if type(n) == float:
        checkFloat = True
    
    # Fraction 변환 가능 여부 체크
    try:
        Fraction(n)  # int, float 체크
    except:
        print("[ERROR] Value", schemestr(n), "not a NUMBER.")
        return None

    # 인자가 1개인 경우
    if args == ():
        return -n

    for i in args:
        # 심볼 안에 값 정의되지 않았을 시
        if i == None:
            return None

        # 인자 값 중 NIL 존재 하면 에러
        if type(i) == bool and i == False:
            print("[ERROR] Value", schemestr(i), "not a NUMBER.")
            return None
        
        # float형 체크
        if type(i) == float:
            checkFloat = True
        
        # Fraction 변환 가능 여부 체크
        try:
            Fraction(i)  # int, float 체크
        except:
            print("[ERROR] Value", schemestr(i), "not a NUMBER.")
            return None
    
    # 정상 작동 경우
    if checkFloat == True:
        return float(n - reduce(op.add, args))
    else:
        return n - reduce(op.add, args)

### / (DIV) 함수 정의
숫자 간 나눗셈 연산을 수행하는 함수이다. 인자 n과 가변인자 args을 입력받고 연산 결과(int/float/Fraction)를 반환한다.
- **인자가 0개인 경우**: Error
- **인자가 1개인 경우**: (1 / 인자) Return
- **인자가 2개 이상인 경우**: 첫번째 인자에서 나머지 인자들의 곱을 나눗셈한 값 Return  

나눗셈의 경우, 0으로 나눴을 때의 오류를 따로 처리해주어야 한다.
- **0으로 나누는 경우**: Error

In [16]:
# DIV 정의
def DIV_func(n, *args):
    checkFloat = False

    # 심볼 안에 값 정의되지 않았을 시
    if n == None or None in args:
        return None

    # 인자 값 중 NIL 존재 하면 에러
    if type(n) == bool and n == False:
        print("[ERROR] Value", schemestr(n), "not a NUMBER.")
        return None

    # Float인 경우 연산 결과 Float 출력
    if type(n) == float:
        checkFloat = True

    # 숫자 여부 체크
    try:
        Fraction(n)  # int, float 체크
    except:
        print("[ERROR] Value", schemestr(n), "not a NUMBER.")
        return None

    # 인자가 1개인 경우
    if args == ():
        # 0으로 나누는 경우
        if n == 0:
            print("[ERROR] Division by ZERO.")
            return None
        # 정상 출력
        else:
            if checkFloat == True:
                return 1 / n
            else:
                return Fraction(1 / n).limit_denominator(2147483647)

    # 인자가 여러개인 경우
    for i in args:
        # 심볼 안에 값 정의되지 않았을 시
        if i == None:
            return None

        # 인자 값 중 NIL 존재 하면 에러
        if type(i) == bool and i == False:
            print("[ERROR] Value", schemestr(i), "not a NUMBER.")
            return None

        # 0으로 나누는 경우
        if i == 0:
            print("[ERROR] Division by ZERO.")
            return None  # 0으로 나누는지 체크

        # float형 체크
        if type(i) == float:
            checkFloat = True
            
        # Fraction 변환 가능 여부 체크
        try:
            Fraction(i)  # int, float 체크
        except:
            print("[ERROR] Value", schemestr(i), "not a NUMBER.")
            return None
        
    # 정상 작동 경우
    if checkFloat == True:
        return float(n / reduce(op.mul, args))
    else:
        return Fraction(n / reduce(op.mul, args)).limit_denominator(2147483647) # int 최대 표현 범위 적용

### = (EqualWith) 함수 정의 (추가 구현 사항)
LISP에선 = 함수와 EQUAL 함수의 역할이 서로 다르다. 그 차이점은 아래와 같다.

- <b>= 의 경우</b> : 숫자만 인자로 받을 수 있다. 가변인자를 받는 함수로 인자의 개수에 제한이 없다.
- **EQUAL의 경우** : 모든 타입을 인자로 받을 수 있다. 단, 2가지 인자에 대한 비교만 가능하다.

숫자 간 동일 여부를 비교하는 함수이다. 가변인자 args을 입력받고 연산 결과(int/float/Fraction)를 반환한다.

- **인자가 0개인 경우**: Error
- **인자가 1개인 경우**: T Return
- **인자가 2개 이상인 경우**: Type과 관계 없이모든 인자들의 값이 동일하면 T Return

In [17]:
# = 정의
def EqualWith_func(*args):
    # 인자 입력이 없을 경우
    if len(args) == 0:
        return None

    # 인자 유효성 체크
    for i in args:
        # 심볼 안에 값 정의되지 않았을 시
        if i == None:
            return None

        # 인자 값 중 NIL 존재 하면 에러
        if type(i) == bool and i == False:
            print("[ERROR] Value", schemestr(i), "not a NUMBER.")
            return None
        
        # Fraction 변환 가능 여부 체크
        try:
            Fraction(i)  # int, float, Fraction 체크
        except:
            print("[ERROR] Value", schemestr(i), "not a NUMBER.")
            return None

    # 다음 인자와의 대소 비교
    for i in range(len(args) - 1):
        if args[i] == args[i + 1]:
            pass
        else:
            return False

    return True

***
## Specification 2 : LISP의 기본 함수

LISP의 기본 함수에 대한 선언 부분이다. List형과 관련된 기본 함수가 주를 이룬다.  

**SETQ**와 **LIST**는 향후 따로 선언하기 때문에 이 부분에서는 생략하도록 하였다.

### CAR 함수 정의
List의 첫번째 원소를 가져오는 함수이다. 가변인자 args을 입력받고 타입과 관계없이 첫번째 원소의 값을 반환한다.

- **인자가 1개가 아닌 경우**: Error

CAR 함수에 적용한 예외 처리(오류 처리)는 다음과 같다.

- **인자가 NIL인 경우**: NIL Return
- **인자가 빈 List인 경우**: NIL Return
- **인자가 List가 아닌 경우**: Error

In [18]:
# CAR 정의
def CAR_func(*args):
    
    # 인자 개수가 맞지 않는 경우
    if len(args[0]) != 1:
        print("[ERROR] Not matched number of args to CAR.")
        return None

    # 선언되지 않은 변수가 대상일 경우
    if args[0][0] is None:
        return None

    # NIL에 대한 조사이거나 빈 리스트인 경우
    elif args[0][0] is False:
        return False

    # 입력 값이 리스트가 아닌 경우
    elif type(args[0][0]) != list:
        print("[ERROR] Value", schemestr(args[0][0]), "not a LIST.")
        return None

    # 리스트의 원소 개수가 0개인 경우 NIL 출력
    elif len(args[0][0]) < 1:
        return False

    # 정상 작동
    else:
        return args[0][0][0]

### CDR 함수 정의
List의 첫번째 원소를 제외한 나머지를 결과로 반환하는 함수이다. 가변인자 args을 입력받고 List를 반환한다.

- **인자가 1개가 아닌 경우**: Error

CDR 함수에 적용한 예외 처리(오류 처리)는 다음과 같다.

- **인자가 NIL인 경우**: NIL Return
- **인자가 빈 List인 경우**: NIL Return
- **인자가 List가 아닌 경우**: Error
- **인자 List의 원소 개수가 0~1개인 경우**: NIL Return

In [19]:
# CDR 정의
def CDR_func(*args):
    
    # 인자 개수가 맞지 않는 경우
    if len(args[0]) != 1:
        print("[ERROR] Not matched number of args to CDR.")
        return None

    # 선언되지 않은 변수가 대상일 경우
    if args[0][0] is None:
        return None

    # NIL에 대한 조사이거나 빈 리스트인 경우
    elif args[0][0] is False:
        return False

    # 입력 값이 리스트가 아닌 경우
    elif type(args[0][0]) != list:
        print("[ERROR] Value", schemestr(args[0][0]), "not a LIST.")
        return None

    # 리스트의 원소 개수가 0~1개인 경우 NIL 출력
    elif len(args[0][0]) < 2:
        return False

    # 정상 작동
    else:
        return args[0][0][1:]

### CADDR 함수 정의
List의 세번째 원소를 반환하는 함수이다. 가변인자 args을 입력받고 타입과 관계없이 첫번째 원소의 값을 반환한다.

- **인자가 1개가 아닌 경우**: Error

CADDR 함수에 적용한 예외 처리(오류 처리)는 다음과 같다.

- **인자가 NIL인 경우**: NIL Return
- **인자가 빈 List인 경우**: NIL Return
- **인자가 List가 아닌 경우**: Error
- **인자 List의 원소 개수가 0~2개인 경우**: NIL Return

In [20]:
# CADDR 정의
def CADDR_func(*args):
    # 인자 개수가 맞지 않는 경우
    if len(args[0]) != 1:
        print("[ERROR] Not matched number of args to CDR.")
        return None

    # 선언되지 않은 변수가 대상일 경우
    if args[0][0] is None:
        return None

    # NIL에 대한 조사이거나 빈 리스트인 경우
    elif args[0][0] is False:
        return False

    # 입력 값이 리스트가 아닌 경우
    elif type(args[0][0]) != list:
        print("[ERROR] Value", schemestr(args[0][0]), "not a LIST.")
        return None

    # 리스트의 원소 개수가 0~2개인 경우 NIL 출력
    elif len(args[0][0]) < 3:
        return False

    # 정상 작동
    else:
        return ((args[0][0][1:])[1:])[0]

### NTH 함수 정의
n번째 원소를 반환하는 함수이다. 인자 n과 가변인자 args을 입력받고 타입과 관계없이 n번째 원소의 값을 반환한다.

- **인자가 2개가 아닌 경우**: Error

NTH 함수에 적용한 예외 처리(오류 처리)는 다음과 같다.

- **첫번째 인자가 정수가 아닌 경우**: Error
- **두번째 인자가 List가 아닌 경우**: Error
- **두번째 인자가 NIL 혹은 빈 List인 경우**: NIL Return
- **첫번째 인자가 두번째 인자의 List의 총 원소 수보다 큰 경우(검색 범위 밖)**: NIL Return

In [21]:
# NTH 정의
def NTH_func(n, *args):
    
    # 인자 개수가 맞지 않는 경우
    if len(args) != 1:
        print("[ERROR] Not matched number of args to NTH.")
        return None

    # 선언되지 않은 변수가 대상일 경우
    if n is None:
        return None

    # 선언되지 않은 변수가 대상일 경우
    if args[0] is None:
        return None

    # 입력 값이 리스트가 아닌 경우
    if type(n) != int:
        print("[ERROR] Value", schemestr(n), "not a INT.")
        return None

    # NIL에 대한 조사이거나 빈 리스트인 경우
    elif args[0] is False:
        return False

    # 입력 값이 리스트가 아닌 경우
    elif type(args[0]) != list:
        print("[ERROR] Value", schemestr(args[0]), "not a LIST.")
        return None

    # 리스트의 원소 개수가 0개이거나 검색 범위 밖일 경우 NIL 출력
    if len(args[0]) < 1 or len(args[0]) < n:
        return False

    # 정상 작동
    else:
        return args[0][n]

### CONS 함수 정의 (추가 기능 구현)
두번째 인자인 기존의 List에 첫번째 인자로 새로운 원소를 추가하여 List를 만든다. 인자 n과 가변인자 args을 입력받고 List를 반환한다.  

- **인자가 2개가 아닌 경우**: Error  

**(추가 기능 구현)** CONS 함수는 두번째 인자가 List가 아니면 첫번째 인자와 두번쨰 인자를 원소로 하는 크기 2짜리 List를 생성한다.  
따라서 두번째 인자가 리스트인 경우 아래의 첫번째 기능, 리스트가 아닌 경우 두번째 기능을 수행한다.  
- **기능1** : 기존의 List의 맨 앞에 새로운 원소 추가
- **기능2** : 첫번째 인자와 두번째 인자를 원소로 하는 새로운 List를 생성

In [22]:
# CONS 정의
def CONS_func(n, *args):
    # 심볼 안에 값 정의되지 않았을 시
    if n == None:
        return None

    if len(args) != 1:
        print("[ERROR] Not matched number of args to CONS.")
        return None

    # 선언되지 않은 변수가 대상일 경우
    if args[0] is None:
        return None

    # 두번째 인자가 리스트가 아닌 경우
    if type(args[0]) != list:
        return [n, args[0]]
    
    # 두번째 인자가 리스트인 경우
    else:
        args[0].insert(0, n)

    return args[0]

### REVERSE 함수 정의
주어진 리스트 내에 원소의 순서를 거꾸로 바꾸는 함수이다. 가변인자 args을 입력받고 List를 반환한다.  

- **인자가 1개가 아닌 경우**: Error  

REVERSE 함수에 적용한 예외 처리(오류 처리)는 다음과 같다.

- **인자가 NIL인 경우**: NIL Return
- **인자가 빈 List인 경우**: NIL Return
- **인자가 List가 아닌 경우**: Error

In [23]:
# REVERSE 정의
def REVERSE_func(*args):
    # 인자 개수가 맞지 않는 경우
    if len(args) != 1:
        print("[ERROR] Not matched number of args to REVERSE.")
        return None

    # 심볼 안에 값 정의되지 않았을 시
    if args[0] == None:
        return None

    # NIL에 대한 조사일 경우
    if args[0] == False:
        return False

    # 리스트가 아닐 경우
    if type(args[0]) != List:
        print("[ERROR] Value", schemestr(args[0]), "not a LIST.")
        return None

    # 리스트의 원소 개수가 0개인 경우 NIL 출력
    elif len(args[0]) < 1:
        return False

    args[0].reverse
    
    return args[0]

### APPEND 함수 정의
주어진 여러개의 List를 하나의 List로 만드는 함수이다. 가변인자 args을 입력받고 List를 반환한다.  

- **인자가 없는 경우**: NIL Return  

APPEND 함수에 적용한 예외 처리(오류 처리)는 다음과 같다.

- **인자가 NIL인 경우**: NIL Return
- **인자가 빈 List인 경우**: NIL Return
- **인자가 List가 아닌 경우**: Error

In [24]:
# APPEND 정의
def APPEND_func(*args):
    # 인자가 없는 경우
    if len(args) == 0:
        return False

    temp = []

    # 리스트 타입 체크
    for i in args:
        
        # 빈 리스트나 NIL의 경우 [] 처리
        if i == False:
            i = []
        
        # 인자가 List가 아닌 경우
        if type(i) != List:
            print("[ERROR] Value", schemestr(i), "not a LIST.")
            return None
        
        else:
            temp += i

    return temp

### LENGTH 함수 정의
주어진 여러개의 List를 하나의 List로 만드는 함수이다. 가변인자 args을 입력받고 List를 반환한다.  

- **인자가 1개가 아닌 경우**: NIL Return  

APPEND 함수에 적용한 예외 처리(오류 처리)는 다음과 같다.

- **인자가 NIL인 경우**: NIL Return
- **인자가 빈 List인 경우**: NIL Return
- **인자가 List가 아닌 경우**: Error

In [25]:
# LENGTH 정의
def LENGTH_func(*args):
    if len(args) != 1:
        print("[ERROR] Not matched number of args to LENGTH.")
        return None

    # 심볼 안에 값 정의되지 않았을 시
    if args[0] == None:
        return None

    # NIL에 대한 조사일 경우
    if args[0] == False:
        return 0

    # 리스트가 아닐 경우
    if type(args[0]) != List:
        print("[ERROR] Value", schemestr(args[0]), "not a LIST.")
        return None

    return len(args[0])

### MEMBER 함수 정의
주어진 리스트 내에 어떤 원소가 있는지 확인하는 함수이다. 리스트에서 찾고자 하는 원소부터 리스트 끝까지가 결과 값으로 반환되고, 리스트 내에 찾고자 하는 원소가 없을 경우 NIL이 반환된다.  
인자 t와 가변인자 args를 입력받고 List를 반환한다.

- **인자가 2개가 아닌 경우**: Error

MEMBER 함수에 적용한 예외 처리(오류 처리)는 다음과 같다.

- **두 번째 인자가 빈 List인 경우**: NIL Return
- **두 번째 인자가 List가 아닌 경우**: Error

In [26]:
# MEMBER 정의
def MEMBER_func(t, *args):
    # 인자 개수가 맞지 않는 경우
    if len(args) != 1:
        print("[ERROR] Not matched number of args to MEMBER.")
        return None

    # 선언되지 않은 변수가 대상일 경우
    if t is None or args[0] is None:
        return None

    # 두번째 인자가 NIL인 경우
    if type(args[0]) == bool and args[0] == False:
        return False

    # 입력 값이 리스트가 아닌 경우
    if type(args[0]) != list:
        print("[ERROR] Value", schemestr(args[0]), "not a LIST.")
        return None

    # 첫번째 인자가 NIL인 경우
    if t is False or args[0] is False:
        return False

    # 정상 작동
    if t in args[0]:
        i = args[0].index(t)
        return args[0][i:]
    else:
        return 'NIL'

### ASSOC 함수 정의
리스트를 원소로 갖는 리스트에서 원소 리스트의 첫번째 원소를 데이터베이스에서의 KEY처럼 사용하여 원하는 리스트를 찾을 수 있는 함수이다.   
인자 t와 가변인자 args를 입력받고 List를 반환한다.

- **인자가 2개가 아닌 경우**: Error

ASSOC 함수에 적용한 예외 처리(오류 처리)는 다음과 같다.

- **인자의 개수가 맞지 않는 경우**: Error
- **인자가 모두 NIL인 경우**: NIL Return
- **인자가 선언되지 않은 변수일 경우**: Error
- **인자가 List가 아닌 경우**: Error
- **두 번째 인자에 key를 원소로 가지는 리스트가 없는 경우**: NIL Return

In [27]:
# ASSOC 정의
def ASSOC_func(t, *args):
    
    # 인자 개수가 맞지 않는 경우
    if len(args) != 1:
        print("[ERROR] Not matched number of args to ASSOC.")
        return None

    # 선언되지 않은 변수가 대상일 경우
    if t is None or args[0] is None:
        return None

    # 입력 값이 리스트가 아닌 경우
    if type(args[0]) != list:
        print("[ERROR] Value", schemestr(args[0]), "not a LIST.")
        return None
    
    # KEY값 Search용 변수
    check_key = 0 
    
    # 정상 작동
    for i in args[0]:
        # 2차원 리스트 여부 확인
        if type(i) != List:
            print("[ERROR] Value", schemestr(i), "not a LIST.")
            return None

        # 대응되는 키 값이 존재할 경우 카운팅
        if i[0] == t:
            check_key = check_key + 1
            return i
        
    if check_key == 0:  # 찾는 원소 가지는 리스트가 없으면 NIL 반환
        return False

### REMOVE 함수 정의
두 번째 인자로 받는 리스트에서 첫 번째 인자와 같은 원소를 찾아 모두 제거하는 함수이다.   
결과값으로 List를 새로 저장하지는 않는다. 인자 t와 가변인자 args를 입력받고 List를 반환한다.

- **인자가 2개가 아닌 경우**: Error

REMOVE 함수에 적용한 예외 처리(오류 처리)는 다음과 같다.  

- **두번째 인자가 빈 List 혹은 NIL인 경우**: NIL Return
- **인자가 List가 아닌 경우**: Error

In [28]:
# REMOVE 정의
def REMOVE_func(t, *args):
    
    # 인자 개수가 맞지 않는 경우
    if len(args) != 1:
        print("[ERROR] Not matched number of args to MEMBER.")
        return None

        # 선언되지 않은 변수가 대상일 경우
    if t is None or args[0] is None:
        return None

    if args[0] == False:
        return False

        # 입력 값이 리스트가 아닌 경우
    elif type(args[0]) != list:
        print("[ERROR] Value", schemestr(args[0]), "not a LIST.")
        return None

    temp = []
    for i in args[0]:
        if i == t:
            pass
        else:
            temp.append(i)

    return temp

### SUBST함수 정의
세 번째 인자에서 두 번째 인자를 찾아 첫 번째 인자로 대치한다. 인자로 t1, t2, 가변인자 args를 받아 List를 반환한다.

- **인자가 3개가 아닌 경우**: Error

SUBST 함수에 적용한 예외 처리(오류 처리)는 다음과 같다.

- **세번째 인자가 List가 아닌 경우**: Error
- **대상 List가 NIL인 경우**: NIL Return

In [29]:
def SUBST_func(*args):
    
    # 인자 개수가 맞지 않는 경우
    if len(args) != 3:
        print("[ERROR] Not matched number of args to SUBST.")
        return None

    # 선언되지 않은 변수가 대상일 경우
    if args[0] is None or args[1] is None or args[2] is None:
        return None

    # 세 번째 인자가 NIL인 경우 NIL 반환
    if args[2] == False:
        return False

    # 입력 값이 리스트가 아닌 경우
    elif type(args[2]) != list:
        print("[ERROR] Value", schemestr(args[2]), "not a LIST.")
        return None

    # 정상 작동
    try:
        args[2][args[2].index(args[1])] = args[0]
        return args[2]
    
    # t2 값이 리스트에 존재하지 않는 경우
    except:
        return args[0]

### PRINT 함수 정의
LISP 내 PRINT 함수는 아래의 두 가지 기능을 포함하고 있다.  
- **기능1** : Console에 인자로 받은 값을 출력(Print)한다.
- **기능2** : 인자로 받은 값을 반환(Return)한다.

인자 n과 가변인자 args을 입력받고 List를 반환한다. 

- **인자가 1개가 아닌 경우**: Error  

schemestr() 함수는 아래 새롭게 정의하며 설명을 추가하도록 한다.

In [30]:
# PRINT 정의
def PRINT_func(*args):
    # 인자 개수가 맞지 않는 경우
    if len(args) != 1:
        print("[ERROR] Not matched number of args to PRINT.")
        return None

    # 선언되지 않은 변수가 대상일 경우
    if args[0] is None:
        return None
    
    # Python 자료형대로 출력되지 않도록 Convert 작업 수행
    print(schemestr(args[0]))
    
    return args[0]

***

## Specification 3 : LISP의 Predicate 함수

LISP의 Predicate 함수에 대한 선언 부분이다. LISP에서 Boolean 값은 아래와 같이 반환된다.
- **T** : 다른 언어의 True 혹은 1과 대응된다.
- **NIL** : 다런 언어의 False 혹은 0과 대응된다.  

Type 확인과 데이터 비교가 주를 이룬다.

### ATOM 함수 정의
대상 값이 ATOM(심볼)일 때만 T(True)를 반환하는 함수이다. 가변인자 args를 입력받아 Bool값(T 혹은 NIL)을 반환한다.  
LISP에서는 List가 아닌 모든 자료형을 ATOM으로 판단하기 때문에 대상 값이 List 자로형에 해당되는지에 대한 체크를 진행한다.

- **인자가 1개가 아닌 경우**: Error

In [31]:
# ATOM 정의
def ATOM_func(*args):
    
    # 인자 개수가 맞지 않는 경우
    if len(args) != 1:
        print("[ERROR] Not matched number of args to ATOM.")
        return None

    # 선언되지 않은 변수가 대상일 경우
    if args[0] is None:
        return None
    
    # List형이 아니면 ATOM형임
    elif not isinstance(args[0], List):
        return True
    
    else:
        return False

### NULL 함수 정의
대상 값이 NIL일 때만 T(True)를 반환하는 함수이다. 가변인자 args를 입력받아 Bool값(T 혹은 NIL)을 반환한다.

- **인자가 1개가 아닌 경우**: Error

In [32]:
# NULL 정의
def NULL_func(*args):
    
    # 인자 개수가 맞지 않는 경우
    if len(args) != 1:
        print("[ERROR] Not matched number of args to NULL.")
        return None

    # 선언되지 않은 변수가 대상일 경우
    if args[0] is None:
        return None

    # 정상 작동 경우
    if args[0] == False:
        return True

    else:
        return False

### NUMBERP 함수 정의
대상 값이 숫자일 때만 T(True)를 반환하는 함수이다. LISP에서 숫자는 정수(int형), 실수(float형), 분수(Fraction) 총 3가지로 표현된다.  
이 때, 숫자 여부의 판단은 Fraction() 함수를 이용함으로써 정수와 실수 이외의 분수도 찾을 수 있도록 한다.  


가변인자 args를 입력받아 Bool값(T 혹은 NIL)을 반환한다.

- **인자가 1개가 아닌 경우**: Error

In [33]:
# NUMBERP 정의
def NUMBERP_func(*args):

    # 인자 개수가 맞지 않는 경우
    if len(args) != 1:
        print("[ERROR] Not matched number of args to NUMBERP.")
        return None

    # 선언되지 않은 변수가 대상일 경우
    if args[0] is None:
        return None
    
    try:
        # NIL값인 경우 NIL 반환
        if args[0] == False and type(args[0]) == bool:
            return False
        
        # Fraction()을 통해 숫자 여부 판단
        else:
            Fraction(args[0])  # int, float 체크
            return True
        
    except:
        return False

### ZEROP 함수 정의
대상 값이 0일 때 T(True)를 반환하는 함수이다. 가변인자 args를 입력받아 Bool값(T 혹은 NIL)을 반환한다.

- **인자가 1개가 아닌 경우**: Error

ZEROP 함수에 적용한 예외 처리(오류 처리)는 다음과 같다.

- **대상 비교값이 NIL인 경우** : Error
- **대상 비교값이 숫자가 아닌 경우** : Error

In [34]:
# ZEROP 정의
def ZEROP_func(*args):
    
    # 인자 개수가 맞지 않는 경우
    if len(args) != 1:
        print("[ERROR] Not matched number of args to ZEROP.")
        return None

    # 선언되지 않은 변수가 대상일 경우
    if args[0] is None:
        return None

    try:
        # 대상 비교값이 NIL인 경우
        if type(args[0]) == bool and args[0] == False:
            print("[ERROR] Value", schemestr(args[0]), "not a NUMBER.")
            return None
        
        # 숫자 여부 체크
        else:
            Fraction(args[0])  # int, float 체크
            if args[0] == 0:
                return True
            else:
                return False
            
    # 대상 비교값이 숫자가 아닌 경우  
    except:
        print("[ERROR] Value", schemestr(args[0]), "not a NUMBER.")
        return None

### MINUSP 함수 정의
대상 값이 음수일 때 T(True)를 반환하는 함수이다. 가변인자 args를 입력받아 Bool값(T 혹은 NIL)을 반환한다.

- **인자가 1개가 아닌 경우**: Error

MINUSP 함수에 적용한 예외 처리(오류 처리)는 다음과 같다.

- **대상 비교값이 NIL인 경우** : Error
- **대상 비교값이 숫자가 아닌 경우** : Error

In [35]:
# MINUSP 정의
def MINUSP_func(*args):
    
    # 인자 개수가 맞지 않는 경우
    if len(args) != 1:
        print("[ERROR] Not matched number of args to MINUSP.")
        return None

    # 선언되지 않은 변수가 대상일 경우
    if args[0] is None:
        return None

    try:
        # 대상 비교값이 NIL인 경우
        if type(args[0]) == bool and args[0] == False:
            print("[ERROR] Value", schemestr(args[0]), "not a NUMBER.")
            return None
        
        # 숫자 여부 체크
        else:
            Fraction(args[0])  # int, float 체크
            if args[0] < 0:
                return True
            else:
                return False
        
    # 대상 비교값이 숫자가 아닌 경우
    except:
        print("[ERROR] Value", schemestr(args[0]), "not a NUMBER.")
        return None

### EQUAL 함수 정의
위에서 언급했듯, LISP에선 = 함수와 EQUAL 함수의 역할이 서로 다르다. 그 차이점은 아래와 같다.

- <b>= 의 경우</b> : 숫자만 인자로 받을 수 있다. 가변인자를 받는 함수로 인자의 개수에 제한이 없다.
- **EQUAL의 경우** : 모든 타입을 인자로 받을 수 있다. 단, 2가지 인자에 대한 비교만 가능하다.

입력 받은 두 인자 값과 Type이 모두이 동일한 경우 T(True)를 반환하는 함수이다. 가변인자 args를 입력받아 Bool값(T 혹은 NIL)을 반환한다.
- **인자가 1개가 아닌 경우**: Error

In [36]:
# EQUAL 정의
def EQUAL_func(n, *args):
    
    # 인자 개수가 맞지 않는 경우
    if len(args) != 1:
        print("[ERROR] Not matched number of args to EQUAL.")
        return None

    # 선언되지 않은 변수가 대상일 경우
    if n is None or args[0] is None:
        return None
    
    # 타입과 값이 모두 일치해야 한다.
    if type(n) == type(args[0]) and n == args[0]:
        return True
    else:
        return False

### > (GreaterThan) 함수 정의
숫자 간 비교 연산을 수행하는 함수이다. 먼저 입력된 인자가 나중에 입력된 인자보다 큰 경우 T(True)를 반환하는 함수다.  
가변인자 args를 입력받아 Bool값(T 혹은 NIL)을 반환한다.

- **인자가 0개인 경우** : Error
- **인자가 1개인 경우** : T Return
- **인자가 2개 이상인 경우** : 먼저 입력된 인자가 나중에 입력된 인자보다 큰 경우 T Return

GreaterThan 함수에 적용한 예외 처리(오류 처리)는 다음과 같다.

- **인자 중 NIL이 존재하는 경우** : Error
- **인자 중 숫자가 아닌 값이 존재하는 경우** : Error

In [37]:
# > 정의
def GreaterThan_func(*args):  # >

    # 인자 입력이 없을 경우
    if len(args) == 0:
        return None

    # 인자 유효성 체크
    for i in args:
        
        # 심볼 안에 값 정의되지 않았을 시
        if i == None:
            return None

        # 인자 값 중 NIL 존재 하면 에러
        if type(i) == bool and i == False:
            print("[ERROR] Value", schemestr(i), "not a NUMBER.")
            return None

        try:
            Fraction(i)  # int, float, Fraction 체크
        except:
            print("[ERROR] Value", schemestr(i), "not a NUMBER.")
            return None

    # 다음 인자와의 대소 비교
    for i in range(len(args) - 1):
        if args[i] > args[i + 1]:
            pass
        else:
            return False

    return True

### < (LessThan) 함수 정의
숫자 간 비교 연산을 수행하는 함수이다. 먼저 입력된 인자가 나중에 입력된 인자보다 작은 경우 T(True)를 반환하는 함수다.  
가변인자 args를 입력받아 Bool값(T 혹은 NIL)을 반환한다.

- **인자가 0개인 경우** : Error
- **인자가 1개인 경우** : T Return
- **인자가 2개 이상인 경우** : 먼저 입력된 인자가 나중에 입력된 인자보다 작은 경우 T Return

LessThan 함수에 적용한 예외 처리(오류 처리)는 다음과 같다.

- **인자 중 NIL이 존재하는 경우** : Error
- **인자 중 숫자가 아닌 값이 존재하는 경우** : Error

In [38]:
# < 정의
def LessThan_func(*args):  # <

    # 인자 입력이 없을 경우
    if len(args) == 0:
        return None

    # 인자 유효성 체크
    for i in args:
        # 심볼 안에 값 정의되지 않았을 시
        if i == None:
            return None

        # 인자 값 중 NIL 존재 하면 에러
        if type(i) == bool and i == False:
            print("[ERROR] Value", schemestr(i), "not a NUMBER.")
            return None

        try:
            Fraction(i)  # int, float, Fraction 체크
        except:
            print("[ERROR] Value", schemestr(i), "not a NUMBER.")
            return None

    # 다음 인자와의 대소 비교
    for i in range(len(args) - 1):
        if args[i] < args[i + 1]:
            pass
        else:
            return False

    return True

### >= (GreaterThanOrEqual) 함수 정의
숫자 간 비교 연산을 수행하는 함수이다. 먼저 입력된 인자가 나중에 입력된 인자보다 크거나 같은 경우 T(True)를 반환하는 함수다.  
가변인자 args를 입력받아 Bool값(T 혹은 NIL)을 반환한다.

- **인자가 0개인 경우** : Error
- **인자가 1개인 경우** : T Return
- **인자가 2개 이상인 경우** : 먼저 입력된 인자가 나중에 입력된 인자보다 크거나 같은 경우 T Return

GreaterThanOrEqual 함수에 적용한 예외 처리(오류 처리)는 다음과 같다.

- **인자 중 NIL이 존재하는 경우** : Error
- **인자 중 숫자가 아닌 값이 존재하는 경우** : Error

In [39]:
# >= 정의
def GreaterThanOrEqual_func(*args):
    # 인자 입력이 없을 경우
    if len(args) == 0:
        return None

    # 인자 유효성 체크
    for i in args:
        # 심볼 안에 값 정의되지 않았을 시
        if i == None:
            return None

        # 인자 값 중 NIL 존재 하면 에러
        if type(i) == bool and i == False:
            print("[ERROR] Value", schemestr(i), "not a NUMBER.")
            return None

        try:
            Fraction(i)  # int, float, Fraction 체크
        except:
            print("[ERROR] Value", schemestr(i), "not a NUMBER.")
            return None

    # 다음 인자와의 대소 비교
    for i in range(len(args) - 1):
        if args[i] >= args[i + 1]:
            pass
        else:
            return False

    return True

### <= (LessThanOrEqual) 함수 정의
숫자 간 비교 연산을 수행하는 함수이다. 먼저 입력된 인자가 나중에 입력된 인자보다 작거나 같은 경우 T(True)를 반환하는 함수다.  
가변인자 args를 입력받아 Bool값(T 혹은 NIL)을 반환한다.

- **인자가 0개인 경우** : Error
- **인자가 1개인 경우** : T Return
- **인자가 2개 이상인 경우** : 먼저 입력된 인자가 나중에 입력된 인자보다 작거나 같은 경우 T Return

LessThanOrEqual 함수에 적용한 예외 처리(오류 처리)는 다음과 같다.

- **인자 중 NIL이 존재하는 경우** : Error
- **인자 중 숫자가 아닌 값이 존재하는 경우** : Error

In [40]:
# <= 정의
def LessThanOrEqual_func(*args):
    # 인자 입력이 없을 경우
    if len(args) == 0:
        return None

    # 인자 유효성 체크
    for i in args:
        # 심볼 안에 값 정의되지 않았을 시
        if i == None:
            return None

        # 인자 값 중 NIL 존재 하면 에러
        if type(i) == bool and i == False:
            print("[ERROR] Value", schemestr(i), "not a NUMBER.")
            return None

        try:
            Fraction(i)  # int, float, Fraction 체크
        except:
            print("[ERROR] Value", schemestr(i), "not a NUMBER.")
            return None

    # 다음 인자와의 대소 비교
    for i in range(len(args) - 1):
        if args[i] <= args[i + 1]:
            pass
        else:
            return False

    return True

### STRINGP 함수 정의
대상 값이 문자열일 때 T(True)를 반환하는 함수이다. 가변인자 args를 입력받아 Bool값(T 혹은 NIL)을 반환한다.  
Atom으로 정의된 자료형 중 " 로 시작하는 데이터를 String으로 간주하여 판단할 수 있다.

- **인자가 1개가 아닌 경우**: Error

STRINGP 함수에 적용한 예외 처리(오류 처리)는 다음과 같다.

In [41]:
# STRINGP 정의
def STRINGP_func(*args):
    
    # 인자 개수가 맞지 않는 경우
    if len(args) != 1:
        print("[ERROR] Not matched number of args to STRINGP.")
        return None

    # 선언되지 않은 변수가 대상일 경우
    if args[0] is None:
        return None
    
    if isinstance(args[0], str) and args[0][0] == '"':
        return True
    else:
        return False

***

## LISP 표준 함수 정의

본 LISP Interpreter는 전역 환경에 함수와 그의 기능, 혹은 Symbol과 그 값을 저장한다. 그 표준 함수가 정의된 환경을 정의하는 함수가 **standard_env()**이다. 


전역 환경은 Dictionary 자료 구조를 가진다. Key 값에 함수 이름이 삽입되고, Value 값에 함수 내용이 lambda 식으로 삽입된다.  


전역 환경의 update는 아래와 같이 수행된다.  
- math 모듈의 함수, 연산자 update
- 위에 정의한 함수들을 실행할 수 있는 lambda 표현식 update

In [42]:
# 표준 함수 정의
def standard_env():
    env = Env()
    env.update(vars(math))
    env.update({
        ############ 1. 사칙 연산 ################
        "+": lambda *args: ADD_func(*args),
        "-": lambda n, *args: SUB_func(n, *args),
        "*": lambda *args: MUL_func(*args),
        "/": lambda n, *args: DIV_func(n, *args),
        "=": lambda *x: EqualWith_func(*x),

        ########### 2. LISP의 기본 함수 ###########
        # SETQ: 아래 구현 완료
        "LIST": lambda *x: None if None in x else False if len(x) == 0 else List(x),
        "CAR": lambda *x: CAR_func(x),
        "CDR": lambda *x: CDR_func(x),
        "CADDR": lambda *x: CADDR_func(x),
        "NTH": lambda n, *x: NTH_func(n, *x),
        "CONS": lambda n, *x: CONS_func(n, *x),
        "REVERSE": lambda *x: REVERSE_func(*x),
        "APPEND": lambda *x: APPEND_func(*x),
        "LENGTH": lambda *x: LENGTH_func(*x),
        "MEMBER": lambda i, *x: MEMBER_func(i, *x),
        "ASSOC": lambda i, *x: ASSOC_func(i, *x),
        "REMOVE": lambda i, *x: REMOVE_func(i, *x),
        "SUBST": lambda *x: SUBST_func( *x),

        ##### 3. LISP의 Predicate 함수 ############
        "ATOM": lambda *x: ATOM_func(*x),
        "NULL": lambda *x: NULL_func(*x),
        "NUMBERP": lambda *x: NUMBERP_func(*x),
        "ZEROP": lambda *x: ZEROP_func(*x),
        "MINUSP": lambda *x: MINUSP_func(*x),
        "EQUAL": lambda i, *x: EQUAL_func(i, *x),
        ">": lambda *x: GreaterThan_func(*x),
        "<": lambda *x: LessThan_func(*x),
        ">=": lambda *x: GreaterThanOrEqual_func(*x),
        "<=": lambda *x: LessThanOrEqual_func(*x),
        "STRINGP": lambda *x: STRINGP_func(*x),
        "PRINT": lambda *x: PRINT_func(*x),
        
        ############ 4. LISP의 조건문 ############
        # IF문: 아래 구현 완료
        # COND문: 아래 구현 완료

    })
    return env

**standard_env()**에서 정의한 환경을 전역 변수로 사용하기 위해 할당한다.

In [47]:
# 전역 변수 환경설정
global_env = standard_env()

LISP 에서 * 은 이전에 Return한 값을 계속하여 저장하며 Update된다. 이를 위해 전역 환경 변수에 미리 초기화를 시켜놓는다.  
<b>LISP 에서 *  Symbol의 초기값은 NIL이다.</b>

In [48]:
# * Symbol 초기화
global_env['stars'] = False

***

## 재귀 방식 Evaluator 구현

LISP는 재귀를 사용한 프로그래밍이 유연한 언어이다. Parser 기본 동작 원리에 따라 재귀 방식의 Evaluate가 가능하도록 설계한다.  

Operator 종류에 따라 그 역할을 지정하여 Program이 수행할 작업을 지시한다.  

- **NIL** : NIL(False)를 반환한다.
- **Symbol** : 심볼에 들어간 값을 반환한다.
- **QUOTE** : 전처리된 Token에서 QUOTE Token을 발견하면 수행, 인용 처리를 한다.  
- **그 외** : 자료형에 따라 그 값을 재귀 방식으로 Evaluate한 값을 반환하거나 그대로 반환한다.  


    이 때, 기본 Evaluate에 적용한 예외 처리(오류 처리)는 다음과 같다.  
  
    - 문자형이 2개 이상의 문자를 가진 경우 : Error
    - 전역 환경 변수에 정의되지 않은 심볼의 경우 : Error
   


***
### Specification 4 : LISP의 조건문  
- **IF** : LISP의 조건문 중 하나이다. 인자의 개수는 2개 혹은 3개이다.
    - 인자가 2개인 경우 : 첫번째 인자가 T를 반환할 시 두번째 인자의 배정문 수행
    - 인자가 3개인 경우 : 첫번째 인자가 T를 반환하면 두번째 인자의 배정문 수행, NIL을 반환하면 세번째인자의 배정문 수행 
    
    IF 에 적용한 예외 처리(오류 처리)는 다음과 같다.  
    

        - 인자의 개수가 1 이하 4이상인 경우 : Error  
    
      
- **COND** : LISP의 조건문 중 하나이다. 인자는 무한할 수 있다.
    - 먼저 입력된 인자부터 탐색하며 조건을 충족하는 경우 배정문을 수행한다.
    - 먼저 입력된 인자부터 탐색하므로 두 개 이상의 조건문을 만족해도 앞서 입력된 인자의 조건문에 대응되는 배정문이 수행된다.
    - 조건문과 그 수행문은 각각이 리스트를 이루어 하나의 인자를 이룬다.  

***
    
- **SETQ** : 심볼(Symbol)에 값을 저장(Binding)한다. 첫번째 인자의 값을 심볼로 지정하여 두번째 인자에서 반환된 값을 저장한다.

    SETQ 에 적용한 예외 처리(오류 처리)는 다음과 같다.  
    

        - 인자의 개수가 2개가 아닌 경우 : Error  
        - 첫번째 인자가 NIL인 경우 : Error
        - 첫번째 인자가 문자형인 경우 : Error
        - 첫번째 인자가 숫자형인 경우 : Error
        - 첫번째 인자가 문자열인 경우 : Error
        - 첫번째 인자가 인용처리인 경우 : Error  
        
- **STAR** : 전역 환경 변수에 저장되어있는 심볼의 값을 리턴한다.  

  

- **그 외** : 재귀 방식을 통해 evaluate() 함수를 계속 실행하여 파싱 후 동작을 실행한다.  
이 때, ( Token 뒤에 미리 선언되지 않은 문자열 Token이 존재할 경우 아래의 예외 처리(오류 처리)를 수행한다.  
  
    - 선언되지 않은 함수의 경우 : Error



In [ ]:
def evaluate(x, env=global_env):
    
    # NIL은 False 반환
    if x == 'NIL':
        return False

    try:
        # 문자형 체크
        if x[0:2] == "#\\":
            
            # 문자형이 2개 이상의 문자를 가질 경우 에러 출력
            if len(x) > 3:
                print("[ERROR] Unrecognized Character Name:", x[2:])
                return None
            
            # 값 그대로 반환
            else:
                return x
    except:
        pass

    if isinstance(x, Letter):
        # 문자열형 리턴
        if x[0] == '"' and x[-1] == '"':
            return x
        
        # Fraction 동작 여부 체크
        try:
            return Fraction(x)
        
        except:
            
            # 심볼인 경우 저장된 값 반환
            if x in env:
                return env[x]
                           
            # 선언되지 않은 심볼일 경우 에러 출력
            else:
                print("[ERROR] Undefined Variable:", x)
                return None
    
    # List 자료형이 아닌 경우 값 그대로 리턴
    elif not isinstance(x, List):
        return x

    op, *args = x

    # 사용되는 Operator 리스트 정의
    op_list = ['+', '-', '*', '/', '=', 'SETQ', 'LIST', 'CAR', 'CDR',
               'CADDR', 'NTH', 'CONS', 'REVERSE', 'APPEND', 'LENGTH',
               'MEMBER', 'ASSOC', 'REMOVE', 'SUBST', 'ATOM', 'NULL',
               'NUMBERP', 'ZEROP', 'MINUSP', 'EQUAL', '<', '<=',
               '>', '>=', 'STRINGP', 'IF', 'COND', 'PRINT']


    # 인용 QUOTE
    if op == 'QUOTE':
        try:
            if len(args[0]) != 0:
                # 문자형의 경우
                if type(args[0]) == str and args[0][0] == '\\':
                    return args[0][1:]
                # 그 외 경우
                else:
                    return args[0]
            # NIL 반환
            else:
                return False
            
        # 리스트 인용이 아닌 경우
        except:
            return args[0]

    elif op == 'STAR':
        return env['stars']

    # 조건문 IF
    elif op == 'IF':
        # 인자의 개수가 맞지 않는 경우 에러
        if len(args) > 3 or len(args) < 2:
            print("[ERROR] Not matched number of args to IF.")
            return None
        
        # 인자의 개수가 2개인 경우
        elif len(args) == 2:
            (test, conseq) = args
            exp = (conseq if evaluate(test, env) or test == 0 else False)
            return evaluate(exp, env)
        
        # 인자의 개수가 3개인 경우
        elif len(args) == 3:
            (test, conseq, alt) = args
            exp = (conseq if evaluate(test, env) or test == 0 else alt)
            return evaluate(exp, env)

    # 조건문 COND
    elif op == 'COND':
        # 인자 0개인 경우 NIL 반
        if len(args) == 0:
            return False

        for i in range(len(args)):
            (test, conseq) = args[i]
            if (evaluate(test, env) or test == 0) == True:
                exp = conseq
                return evaluate(exp, env)
            else:
                continue
        return False

    # 할당 SETQ
    elif op == 'SETQ':
        
        if len(args) == 0:
            return False
        try:
            # 인자 리스트를 심볼과 표현식으로 분리
            (symbol, exp) = args

        except:
            # 인자 개수가 맞지 않을 경우
            print("[ERROR] Not matched number of args to SETQ.")
            return None

        try:
            #eval()은 연산 가능한 수식을 의미하므로 숫자형 필터링에 활용
            eval(str(symbol))
            
            print("[ERROR] Variable name is not a symbol:", schemestr(evaluate(symbol)))
            return None
        
        except:
            try:
                # 문자형의 경우
                if symbol[0:2] == "#\\":
                    print("[ERROR] Variable name is not a symbol:", schemestr(evaluate(symbol)))
                    return None
                
                # NIL 값의 경우
                elif symbol == "NIL":
                    print("[ERROR] NIL is a constant and thus can't be set.")
                    return None
                
                # 재귀 방식의 Evaluate
                env[symbol] = evaluate(exp, env)
                
            except:
                # 선언되지 않은 변수일 경우
                if type(exp) == str and exp[0] not in ['#', "'", '"']:
                    print("[ERROR] Undefined variable:", exp)
                    return None

            return env[symbol]
    
    # 심볼 STAR
    elif op == 'STAR':
        return env['stars']

    # 그 외 경우
    else:
        # Operator 목록에 없는 경우
        if op not in op_list:
            print("[ERROR] Undefined function", str(op) + ".")
            return None
        
        # 인자가 없어도 에러메시지를 출력하지 않는 경우를 제외하곤 에러메시지 출력
        if len(args) == 0 and op not in ['+', '*', 'SETQ', 'LIST', 'APPEND', 'COND']:
            print("[ERROR] Not matched number of args to", str(op) + ".")
            return None
    
        # 재귀 방식으로 Evaluate 반복 수행
        proc = evaluate(op, env)
        vals = [evaluate(arg, env) for arg in args]
        
        # 결과 Return
        return proc(*vals)

***

## LISP Interpreter 입출력기

LISP Interpreter는 다음의 특징을 가진다.

- **특징1** : '(' 개수가 ')' 개수보다 많은 입력의 경우, 실행하지 않고 추가 입력을 기다린다.
- **특징2** : '(' 개수가 ')' 개수보다 적은 입력의 경우, 에러 메시지를 출력한다.
- **특징3** : ';' 이후에 쓰이는 문자는 주석 처리 한다.

Interpreter 가독성을 위해 '>>>' 을 매 입력마다 출력하도록 하였고, '특징1' 경우의 출력 대기 시에는 실제 LISP Interpreter와 동일하게 '>>>'을 출력하지 않았다.  

또한 ; 이후의 문자는 주석처리하여 무시하도록 하였다.

Python을 이용하여 LISP Interpreter를 구현하였기 때문에 출력 형식을 LISP에 맞게 Convert하는 작업이 필요하다.  
이는 직접 수정 혹은 shemestr() 함수를 이용하였으며 아래의 경우와 같다.  


- Python의 True : LISP의 T로 출력
- Python의 False : LISP의 NIL로 출력
- 그 외의 경우 : shemestr() 함수를 이용여 결과 Convert

또한, 반환 값을 계속하여 * 심볼에 업데이트하는 작업을 실시하였다. **(추가 구현 사항)**

In [51]:
# main 함수 역할
def repl(prompt='>>> '):
    
    # 새로운 입력을 위한 변수 초기화
    new_line = True
    count_left = 0
    count_right = 0
    userCommand = ""
    
    while True:
        
        # 새로운 입력인 경우
        if new_line == True:
            temp = input(prompt)
            
            # ; 주석 처리
            if ';' in temp:
                i = temp.index(';')
                temp = temp[:i]
            else:
                pass
            
            userCommand += temp
            new_line = False

        # 새로운 입력이 아닌 경우
        elif new_line == False:
            temp = input()
            
            # ; 주석 처리
            if ';' in temp:
                i = temp.index(';')
                temp = temp[:i]
            else:
                pass
            
            userCommand += " " + temp

        count_left += temp.count('(') # '(' 개수 카운트
        count_right += temp.count(')') # ')' 개수 카운트

        # 공백 뿐인 입력인 경우 새로운 입력을 받는다.
        if temp.strip() == "":
            continue

        # '(' 개수가 # ')' 개수보다 적은 입력의 경우 에러 출력
        if count_left < count_right:
            print("[ERROR] Unexpected ')'")
            
            # 새로운 입력을 위한 초기화
            new_line = True
            count_left = 0
            count_right = 0
            userCommand = ""
            continue
        
        # '(' 개수가 # ')' 개수보다 많은 입력의 경우 입력 대기
        if count_left > count_right:
            continue
        
        # LEXER, PARSER 수행 및 Evaluate 수행
        val = evaluate(parse(userCommand))

        # 결과값이 존재하면
        if val is not None:
            
            # Python의 True를 LISP의 T로 출력
            if type(val) == bool and val == True:
                print('T')
                
            # Python의 False를 LISP의 NIL로 출력
            elif type(val) == bool and val == False:
                print('NIL')
                
            # 결과 가공함수를 거쳐 콘솔에 결과 출력
            else:
                print(schemestr(val))
                
            # 리턴값을 * 심볼에 저장
            global_env['stars'] = val
            
        # 새로운 입력을 위한 변수 초기화
        temp = ""
        count_left = 0
        count_right = 0
        userCommand = ""
        new_line = True

***

## 결과 가공 함수

위에서 설명한 바와 같이 Python을 이용하여 LISP Interpreter를 구현하였기 때문에 출력 형식을 LISP에 맞게 Convert하는 작업이 필요하다.  
shemestr() 함수를 이용하여 아래와 같이 Convert하였다.  

- Python의 List 자료구조 : ','를 제거하고 '[', ']' 룰 '(', ')'로 교체하여 출력
- 그 외 : String형으로 변환하여 출력

In [52]:
# 결과 가공 함수
def schemestr(exp):
    if isinstance(exp, List):
        return '(' + ' '.join(map(schemestr, exp)) + ')'
    else:
        if type(exp) == bool and exp == False:
            return 'NIL'
        else:
            return str(exp)

***

## Run LISP Interpreter! : Interactive mode & File execution Mode

두 가지 모드의 LISP Interpreter를 지원한다.

- **Mode1** : Interactive Mode
- **Mode2** : File Execution Mode

코드의 첫 실행 시 모드를 선택할 수 있도록 구현하였으며, 잘못된 입력시 에러 메시지를 출력하고 정상 입력을 받을 때까지 반복한다.

- **Mode1** 진입 시
    main()함수 역할을 수행하는 repl() 함수를 이용하면 LISP Interpreter를 실행할 수 있다.  

- **Mode2** 진입 시
    Code와 동일한 디렉토리에 있는 code.in 파일을 읽어 그 결과를 출력한다.
    
확인되지 않은 다른 에러에 대한 예외 처리를 추가하여 절대 Interpreter가 작동이 중단되는 경우가 없도록 한다.

In [ ]:
while True:
    try:
        mode_select = int(input("Select Mode -> 1.Interactive Mode 2. File execution Mode : "))
    except:
        print("Not Integer Type Input. Retry.")
        continue

    # Interactive Mode
    if mode_select == 1:
        while True:
            try:
                repl()
            except:
                print("[ERROR] Unconfirmed Error")
                continue

    # File execution mode
    elif mode_select == 2:

        print('\nReading file "code.in"... ')
        print('#########################\n')

        try:
            # File INPUT
            f = open('code.in', 'r', encoding='utf-8')
            lines = f.readlines()
        except:
            # File INPUT
            f = open('code.in', 'r')
            lines = f.readlines()

        # 새로운 입력을 위한 변수 초기화
        new_line = True
        count_left = 0
        count_right = 0
        userCommand = ""

        for line in lines:

            # 새로운 입력인 경우
            if new_line == True:
                print('>>> ' + line)
                temp = line

                # ; 주석 처리
                if ';' in temp:
                    i = temp.index(';')
                    temp = temp[:i]
                else:
                    pass

                userCommand += temp
                new_line = False

            # 새로운 입력이 아닌 경우
            elif new_line == False:
                print(line)
                temp = line

                # ; 주석 처리
                if ';' in temp:
                    i = temp.index(';')
                    temp = temp[:i]
                else:
                    pass

                userCommand += " " + temp

            count_left += temp.count('(')  # '(' 개수 카운트
            count_right += temp.count(')')  # ')' 개수 카운트

            # 공백 뿐인 입력인 경우 새로운 입력을 받는다.
            if temp.strip() == "":
                continue

            # '(' 개수가 # ')' 개수보다 적은 입력의 경우 에러 출력
            if count_left < count_right:
                print("[ERROR] Unexpected ')'")

                # 새로운 입력을 위한 초기화
                new_line = True
                count_left = 0
                count_right = 0
                userCommand = ""
                continue

            # '(' 개수가 # ')' 개수보다 많은 입력의 경우 입력 대기
            if count_left > count_right:
                continue

            # '(' 개수가 # ')' 개수보다 적은 입력의 경우 에러 출력
            if count_left < count_right:
                print("[ERROR] Unexpected ')'")

                # 새로운 입력을 위한 초기화
                new_line = True
                count_left = 0
                count_right = 0
                userCommand = ""
                continue

            # 결과 계산
            val = evaluate(parse(userCommand))

            # 결과값이 존재하면
            if val is not None:
                # Python의 True를 LISP의 T로 출력
                if type(val) == bool and val == True:
                    print('T')

                # Python의 False를 LISP의 NIL로 출력
                elif type(val) == bool and val == False:
                    print('NIL')

                # 결과 가공함수를 거쳐 콘솔에 결과 출력
                else:
                    print(schemestr(val))

                    # 리턴값을 * 심볼에 저장
                global_env['stars'] = val

            print()
            new_line = True
            count_left = 0
            count_right = 0
            userCommand = ""

        print('#########################')
        print('LISP Interpreter Done.\n')
        break

    else:
        print("Not expected mode. Retry.")
        continue